In [15]:
import numpy as np
from gurobipy import *
import pandas as pd
from itertools import combinations

In [16]:
def Read_Data():
    
    df_interviewer = pd.read_excel("Data.xlsx",sheet_name="Interviewer Information")
    df_candidate = pd.read_excel("Data.xlsx",sheet_name="Interviewee Information")
    novice = {}
    quality = {}
    max_interview = {}
    i_available = {}
    c_available = {}
    time_col_i = {}
    time_col_c = {}
    times = ["9:00","9:30","10:00","10:30","11:00","11:30"\
             ,"12:00","12:30","13:00","13:30","14:00","14:30"]
    behavioral = {}
    case = {}
    i = 5
    c = 2
    for t in times:
        if t == "9:00":
            time_col_i[t] = "Availability"
            time_col_c[t] = "Unnamed: 1" 
        else:
            time_col_i[t] = "Unnamed: " +str(i)
            time_col_c[t] = "Unnamed: " +str(c)
            i= i+1
            c= c+1
        if "3" in t:
            behavioral[t] = 1
            case[t] = 0
        else:
            behavioral[t] = 0
            case[t] = 1
    for i in range (1,41):
        novice[i] = df_interviewer.loc[i,"Unnamed: 1"]
        quality[i] = df_interviewer.loc[i,"Unnamed: 3"]
        max_interview[i] = df_interviewer.loc[i,"Unnamed: 2"]

    for t in times:
        for c in range(1,21):
            c_available[(c,t)] = df_candidate.loc[c,time_col_c[t]]
        for i in range(1,41):
            i_available[(i,t)] = df_interviewer.loc[i,time_col_i[t]]
                
    return df_interviewer,df_candidate, novice, quality, max_interview, \
            i_available, c_available, times, behavioral, case

In [19]:
def Testing_Constraints():
    
    
    df_interviewer, df_candidate, novice, quality, max_interview, \
    i_available, c_available, times, behavioral, case = Read_Data()
    
    time_index = {}
    for i in range(0,12):
        for t in times:
            time_index[t] = i
    reverse_time_index = {}
    for i in range(0,12):
        for t in times:
            reverse_time_index[i] = t
    
    optimal_solutions = {}
    constraint_num = range(1,10)
    for r in range(1,10):
        for combo in combinations(constraint_num,r):
            m = Model()
            x = {}
            z = {}

            for c in range(1,21):
                for t in times:
                    z[(c,t)] = m.addVar(0,1,1,GRB.BINARY)
                    #z[(c,t)] = 1 if there is an interview conducted for candidate c at time t
                    for i in range(1,41):
                        x[(i,c,t)] = m.addVar(0,1,1,vtype=GRB.BINARY)
                        #x[(i,c,t)] = 1 if there is an interview conducted by interviewer 1 with candidate c at t time

            m.setObjective(quicksum(quality[i]*z[(c,t)]*x[(i,c,t)] for i in range(1,41)\
                                   for c in range(1,21) for t in times),GRB.MAXIMIZE)
            
            #Interviews only assigned if both parties available
            for i in range(1,41):
                for c in range(1,21):
                    for t in times:
                        m.addConstr(x[(i,c,t)]*z[(c,t)] <= i_available[(i,t)]*c_available[(c,t)])
            
            #Each candidate assigned 2 interviews
                for c in range(1,21):
                    m.addConstr(quicksum(z[(c,t)] for t in times) == 2)
                            
            
            constraints_used = ""
            if 1 in combo:
            #Each candidate assigned 2 interviewers
                constraints_used = constraints_used + "1,"
                for c in range(1,21):
                    for t in times:
                        m.addConstr(quicksum(x[(i,c,t)] for i in range(1,41)) == 2*z[(c,t)])
            if 2 in combo:
            #Behavioral and Case back to back
                constraints_used = constraints_used + " 2,"
                for c in range(1,21):
                    for t in times:
                        if time_index[t]%2 == 0:
                            m.addConstr(z[(c,t)] <= z[(c,t+1)])
            if 3 in combo:
            #No interviewer may interview the same candidate twice
                constraints_used = constraints_used + " 3,"
                for i in range(1,41):
                    for c in range(1,21):
                        m.addConstr(quicksum(x[(i,c,t)] for t in times) <= 1)
                        
            if 4 in combo:
            #No interviewer assigned more than their max
                constraints_used = constraints_used + " 4,"
                for i in range(1,41):
                    m.addConstr(quicksum(x[(i,c,t)] for c in range(1,21) \
                                         for t in times) <= max_interview[i])
            if 5 in combo:
            #Novice interviewers cannot conduct case
                constraints_used = constraints_used + " 5,"
                for i in range(1,41):
                    if novice[i] == 1:
                        for t in times:
                            if time_index[t] % 2 == 1:
                                m.addConstr(quicksum(x[(i,c,t)] for c in range(1,21)) == 0)
            if 6 in combo:
                constraints_used = constraints_used + " 6,"
            #Interviewers with multiple interviews cannot have 30 mins
                for time_combo in combinations(reverse_time_index,2):
                    if int(time_combo[1]) - int(time_combo[0]) > 2:
                        for i in range(1,41):
                            m.addConstr(quicksum(x[(i,c,reverse_time_index[time_combo[1]])] for c in range(1,21))* \
                                         quicksum(x[(i,c,reverse_time_index[time_combo[0]])] for c in range(1,21)) == 0)
            if 7 in combo:
                constraints_used = constraints_used + " 7,"
                #Each candidate is assigned less than or equal to 2 interviewers
                for c in range(1,21):
                    for t in times:
                        m.addConstr(quicksum(x[(i,c,t)]*z[(c,t)] for i in range(1,41)) <= 2)
                      
            if 8 in combo:
                constraints_used = constraints_used + " 8,"
                #Interviews are 1 behavioral and 1 case
                for c in range(1,21):
                    m.addConstr(quicksum(z[(c,t)]*behavioral[t] for t in times) == 1)
                    m.addConstr(quicksum(z[(c,t)]*case[t] for t in times) == 1)
                    
    
            if 9 in combo:
                constraints_used = constraints_used + " 9"
                #No more than 10 interviews going on at once
                for t in times:
                    m.addConstr(quicksum(z[(c,t)] for c in range(1,21)) <= 10)
                            
            m.setParam("OutputFlag",False)
            m.optimize()
            if m.status == GRB.OPTIMAL:
                optimal_solutions[constraints_used] = m.objVal                           
    
    return optimal_solutions
    

In [18]:
Testing_Constraints()
#Takes a little while to run

/Users/benpaoni/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/benpaoni/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


{' 2,': 4308.0,
 ' 3,': 2839.0,
 ' 4,': 592.0,
 ' 5,': 3564.0,
 ' 6,': 4264.0,
 ' 7,': 644.0,
 ' 8,': 4148.0,
 ' 10': 4292.0,
 ' 2, 3,': 2839.0,
 ' 2, 4,': 592.0,
 ' 2, 5,': 3564.0,
 ' 2, 6,': 4264.0,
 ' 2, 7,': 644.0,
 ' 2, 8,': 4148.0,
 ' 2, 10': 4292.0,
 ' 3, 4,': 592.0,
 ' 3, 5,': 2318.0,
 ' 3, 6,': 2839.0,
 ' 3, 7,': 634.0,
 ' 3, 8,': 2839.0,
 ' 3, 10': 2839.0,
 ' 4, 5,': 461.0,
 ' 4, 6,': 592.0,
 ' 4, 7,': 458.0,
 ' 4, 8,': 592.0,
 ' 4, 10': 592.0,
 ' 5, 6,': 3536.0,
 ' 5, 7,': 644.0,
 ' 5, 8,': 3377.0,
 ' 5, 10': 3564.0,
 ' 6, 7,': 642.0,
 ' 6, 8,': 4104.0,
 ' 6, 10': 4248.0,
 ' 7, 8,': 627.0,
 ' 7, 10': 644.0,
 ' 8, 10': 4096.0,
 ' 2, 3, 4,': 592.0,
 ' 2, 3, 5,': 2318.0,
 ' 2, 3, 6,': 2839.0,
 ' 2, 3, 7,': 634.0,
 ' 2, 3, 8,': 2839.0,
 ' 2, 3, 10': 2839.0,
 ' 2, 4, 5,': 461.0,
 ' 2, 4, 6,': 592.0,
 ' 2, 4, 7,': 458.0,
 ' 2, 4, 8,': 592.0,
 ' 2, 4, 10': 592.0,
 ' 2, 5, 6,': 3536.0,
 ' 2, 5, 7,': 644.0,
 ' 2, 5, 8,': 3377.0,
 ' 2, 5, 10': 3564.0,
 ' 2, 6, 7,': 642.0,
 ' 2, 6, 8,'

In order to decide which constraints to use, I created a Testing_Constraints function. I first coded the following essential constraints:

    1. Each candidate is assigned two interviews.
    3. An interview can only be conducted if both the candidate and selected interviewer are available.
    
I felt that these constraints were essential to the schedule's functioning, thus they were used in every iteration of the problem. Next, I coded the rest of the constraints that the problem described. I put these additional constraints under a for loop, which iterated over all the possible combinations of constraints. For each combination, the model was optimized. If an optimal solution was found, it was stored in a dictionary where the objective value was the key and outputted the constraints that were used for that iteration. After running the function and seeing which constraints led to optimal solutions, I decided to use the iteration where the most constraints were satisfied. The following constraints, in addition to the five listed above, were used in my final relaxed problem:

    1. Each candidate is assigned 2 interviewers.
    2. Behavioral and Case interviews are back to back.
    3. No interviewer may interview the same candidate twice.
    4. No interviewer assigned more than their max.
    5. Novice interviewers cannot conduct case.
    6. Interviewers with multiple interviews cannot have 30 mins.
    7. Each candidate is assigned less than or equal to 2 interviewers.
    8. Interviews are 1 behavioral and 1 case.
    9. No more than 10 interviews going on at once.
    
The objective value that used the most constraints was 446.0 and used the two essential constraints as well as constraints (2,3,4,5,6,7,8,9), thus, these were all the constraints I used in the final problem.

In [26]:
def Problem():
    
    df_interviewer, df_candidate, novice, quality, max_interview, \
    i_available, c_available, times, behavioral, case = Read_Data()
    
    m = Model()
    x = {}
    z = {}
    time_index = {}
    
    for i in range(0,12):
        for t in times:
            time_index[t] = i
    reverse_time_index = {}
    for i in range(0,12):
        for t in times:
            reverse_time_index[i] = t
    
    for c in range(1,21):
        for t in times:
            z[(c,t)] = m.addVar(0,1,1,GRB.BINARY, name = f"z_{c}_{t}")
            #z[(c,t)] = 1 if there is an interview conducted for candidate c at time t
            for i in range(1,41):
                x[(i,c,t)] = m.addVar(0,1,1,vtype=GRB.BINARY, name = f"x_{i}_{c}_{t}")
                #x[(i,c,t)] = 1 if there is an interview conducted by interviewer 1 with candidate c at t time
    m.setObjective(quicksum(quality[i]*z[(c,t)]*x[(i,c,t)] for i in range(1,41)\
                                   for c in range(1,21) for t in times),GRB.MAXIMIZE)
                
    #Behavioral and Case back to back    
    for c in range(1,21):
        for t in times:
            if time_index[t]%2 == 0:
                m.addConstr(z[(c,t)] <= z[(c,t+1)])
           
    #No interviewer may interview the same candidate twice
    for i in range(1,41):
        for c in range(1,21):
            m.addConstr(quicksum(x[(i,c,t)] for t in times) <= 1)
                        
            
    #No interviewer assigned more than their max
               
    for i in range(1,41):
        m.addConstr(quicksum(x[(i,c,t)] for c in range(1,21) \
                                         for t in times) <= max_interview[i])
            
    #Novice interviewers cannot conduct case
                
    for i in range(1,41):
        if novice[i] == 1:
            for t in times:
                if time_index[t] % 2 == 1:
                    m.addConstr(quicksum(x[(i,c,t)] for c in range(1,21)) == 0)
            
            
    #Interviewers with multiple interviews cannot have 30 mins
    for time_combo in combinations(reverse_time_index,2):
        if int(time_combo[1]) - int(time_combo[0]) > 2:
            for i in range(1,41):
                m.addConstr(quicksum(x[(i,c,reverse_time_index[time_combo[1]])] for c in range(1,21))* \
                                         quicksum(x[(i,c,reverse_time_index[time_combo[0]])] for c in range(1,21)) == 0)
            
                
    #Each candidate is assigned less than or equal to 2 interviewers
    for c in range(1,21):
        for t in times:
            m.addConstr(quicksum(x[(i,c,t)]*z[(c,t)] for i in range(1,41)) <= 2)
                      
            
            
    #Interviews are 1 behavioral and 1 case
    for c in range(1,21):
        m.addConstr(quicksum(z[(c,t)]*behavioral[t] for t in times) == 1)
        m.addConstr(quicksum(z[(c,t)]*case[t] for t in times) == 1)
            

    #Interviews only assigned if both parties available
    for i in range(1,41):
        for c in range(1,21):
            for t in times:
                m.addConstr(x[(i,c,t)]*z[(c,t)] <= i_available[(i,t)]*c_available[(c,t)])
            
    #Each candidate assigned 2 interviews
    for c in range(1,21):
        m.addConstr(quicksum(z[(c,t)] for t in times) == 2)
    
    m.setParam("OutputFlag",False)
    m.optimize()   
    
    interviewer_schedule = "interviewer_schedule.txt"
    candidate_schedule = "candidate_schedule.txt"
    
    with open(interviewer_schedule,"w") as file:
        for v in m.getVars():
            for i in range(1,41):
                for c in range(1,21):
                    for t in times:
                        if v.varName == "x_"+str(i)+"_"+str(c)+"_"+t:
                            if v.X == 1:
#                                 file.write("Interviewer " + str(i) + " has an interview with Candidate " + str(c) + " at " + t + " \n")
                                file.write(t + ": Candidate " + str(c) + " has an interview with Interviewer " + str(i) + " \n")
                                
                                    
    return m.objVal, interviewer_schedule

In [27]:
Problem()
#Takes a couple minutes to run

/Users/benpaoni/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/benpaoni/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


(446.0, 'interviewer_schedule.txt')